In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import spacy

In [2]:
#load dataset
df=pd.read_csv('seminyak_data.csv')
df.head()

,Name,Country,Date,Content
0,MADHURKJAIN,"Vadodara, India",Oct 2019,Pristine white sandy beach. Ideal place to rel...
1,ind,"England, UK",Sep 2019,Arrrrh the beach... this is one place we didn’...
2,PrestonGuild,United Kingdom,Jul 2019,Once you go past Legian beach you are on Semin...
3,Aslam_Sherif,"London, UK",Aug 2019,Seminyak beach is nice and clean. I found it s...
4,Sandy Ta,"Singapore, Singapore",May 2019,Purposely chose Seminyak beach of all places i...


In [3]:
#case folding
df['case_folded'] = df['Content'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
# df['case_folded'].head()
df.drop(["Name", "Country"], axis=1, inplace=True)
df.head()

,Date,Content,case_folded
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...


In [4]:
#emoticon/emoji removal
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['no_emot'] = df['case_folded'].apply(lambda x: remove_emoji(x))

In [5]:
#remove punctuation
x = 'everyne walking arounf had no masks on, i seem...'
# re.sub(r'[^\w ]+', "", x)

df['no_punct'] = df['no_emot'].apply(lambda x: re.sub(r'[^\w ]+', "", x))
df.head()

,Date,Content,case_folded,no_emot,no_punct
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...


In [6]:
#frequent words removal
# text = ' '.join(df['no_punct'])
# text = text.split()

# freq_words = pd.Series(text).value_counts()
# f20 = freq_words[:20]
# f20

# df['no_fr_words'] = df['no_punct'].apply(lambda x: ' '.join([t for t in x.split() if t not in f20]))

cnt = Counter()
for text in df["no_punct"].values:
    for word in text.split():
        cnt[word] += 1
        
# cnt.most_common(10)

FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["no_freq_words"] = df["no_punct"].apply(lambda text: remove_freqwords(text))
df.head()

,Date,Content,case_folded,no_emot,no_punct,no_freq_words
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...,pristine white sandy ideal place relax or swim...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...,arrrrh this one place we didnt list on our age...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once go past legian are on seminyak beachthere...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...,seminyak nice clean i found it some parts very...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak all places bali my tr...


In [7]:
#Stopwords Removal
stop = set(stopwords.words('english'))
# df['no_stopwords'] = df['no_fr_words'].apply(lambda x: ' '.join([t for t in x.split() if t not in stopwords]))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["no_stopwords"] = df["no_freq_words"].apply(lambda text: remove_stopwords(text))

df.head()

,Date,Content,case_folded,no_emot,no_punct,no_freq_words,no_stopwords
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...,pristine white sandy ideal place relax or swim...,pristine white sandy ideal place relax swim lo...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...,arrrrh this one place we didnt list on our age...,arrrrh one place didnt list agenda planning tr...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once go past legian are on seminyak beachthere...,go past legian seminyak beachthere high end ho...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...,seminyak nice clean i found it some parts very...,seminyak nice clean found parts busy loud part...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak all places bali my tr...,purposely chose seminyak places bali trip want...


In [8]:
#rare words removal
# rare20 = most_common.tail(20)
# df['no_rare_words'] = df['no_stopwords'].apply(lambda x : ' '.join([t for t in x.split() if t not in rare20]))

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["no_rare_words"] = df["no_stopwords"].apply(lambda text: remove_rarewords(text))
df.head()

,Date,Content,case_folded,no_emot,no_punct,no_freq_words,no_stopwords,no_rare_words
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...,pristine white sandy ideal place relax or swim...,pristine white sandy ideal place relax swim lo...,pristine white sandy ideal place relax swim lo...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...,arrrrh this one place we didnt list on our age...,arrrrh one place didnt list agenda planning tr...,arrrrh one place didnt list agenda planning tr...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once go past legian are on seminyak beachthere...,go past legian seminyak beachthere high end ho...,go past legian seminyak beachthere high end ho...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...,seminyak nice clean i found it some parts very...,seminyak nice clean found parts busy loud part...,seminyak nice clean found parts busy loud part...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak all places bali my tr...,purposely chose seminyak places bali trip want...,purposely chose seminyak places bali trip want...


In [9]:
#Stemming
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["text_stemmed"] = df["no_rare_words"].apply(lambda text: stem_words(text))
df.head()

,Date,Content,case_folded,no_emot,no_punct,no_freq_words,no_stopwords,no_rare_words,text_stemmed
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...,pristine white sandy ideal place relax or swim...,pristine white sandy ideal place relax swim lo...,pristine white sandy ideal place relax swim lo...,pristin white sandi ideal place relax swim loc...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...,arrrrh this one place we didnt list on our age...,arrrrh one place didnt list agenda planning tr...,arrrrh one place didnt list agenda planning tr...,arrrrh one place didnt list agenda plan trip p...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once go past legian are on seminyak beachthere...,go past legian seminyak beachthere high end ho...,go past legian seminyak beachthere high end ho...,go past legian seminyak beachther high end hot...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...,seminyak nice clean i found it some parts very...,seminyak nice clean found parts busy loud part...,seminyak nice clean found parts busy loud part...,seminyak nice clean found part busi loud part ...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak all places bali my tr...,purposely chose seminyak places bali trip want...,purposely chose seminyak places bali trip want...,purpos chose seminyak place bali trip want lea...


In [10]:
#lemmatization
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["review"] = df["no_rare_words"].apply(lambda text: lemmatize_words(text))
df.head()

,Date,Content,case_folded,no_emot,no_punct,no_freq_words,no_stopwords,no_rare_words,text_stemmed,review
0,Oct 2019,Pristine white sandy beach. Ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach. ideal place to rel...,pristine white sandy beach ideal place to rela...,pristine white sandy ideal place relax or swim...,pristine white sandy ideal place relax swim lo...,pristine white sandy ideal place relax swim lo...,pristin white sandi ideal place relax swim loc...,pristine white sandy ideal place relax swim lo...
1,Sep 2019,Arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach... this is one place we didn’...,arrrrh the beach this is one place we didnt li...,arrrrh this one place we didnt list on our age...,arrrrh one place didnt list agenda planning tr...,arrrrh one place didnt list agenda planning tr...,arrrrh one place didnt list agenda plan trip p...,arrrrh one place didnt list agenda plan trip p...
2,Jul 2019,Once you go past Legian beach you are on Semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once you go past legian beach you are on semin...,once go past legian are on seminyak beachthere...,go past legian seminyak beachthere high end ho...,go past legian seminyak beachthere high end ho...,go past legian seminyak beachther high end hot...,go past legian seminyak beachthere high end ho...
3,Aug 2019,Seminyak beach is nice and clean. I found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean. i found it s...,seminyak beach is nice and clean i found it so...,seminyak nice clean i found it some parts very...,seminyak nice clean found parts busy loud part...,seminyak nice clean found parts busy loud part...,seminyak nice clean found part busi loud part ...,seminyak nice clean find part busy loud part q...
4,May 2019,Purposely chose Seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak beach of all places i...,purposely chose seminyak all places bali my tr...,purposely chose seminyak places bali trip want...,purposely chose seminyak places bali trip want...,purpos chose seminyak place bali trip want lea...,purposely chose seminyak place bali trip want ...


In [11]:
df.drop(["case_folded", "no_emot", "no_punct", "no_freq_words", "no_stopwords", "no_rare_words", "text_stemmed"], axis=1, inplace=True)
df.sample(5)

,Date,Content,review
2735,May 2016,Nice beach once you negotiate the right rate f...,nice negotiate right rate deck chair great day...
2580,Jul 2016,This beach is not clean at all. The best beach...,clean best beach esperance wa whitsundays qld ...
3744,Jan 2015,"Apparently December is the Seminyak Beach, Tra...",apparently december seminyak trash expo annual...
3854,Nov 2014,Beach was very clean and able to walk miles al...,clean able walk mile along need shoe flip flop...
635,Aug 2018,"Perfect place to enjoy the sun, sand and the w...",perfect place enjoy sun sand weather go even 1...


In [12]:
#data labelling
sid = SentimentIntensityAnalyzer()
# df = pd.read_csv('doublesix_lemmatized.csv', sep='\t')
# df.head()

df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment'] = df['compound'].apply(lambda c: '1' if c >= 0 else '0')

# df.sample(20)

df.to_csv('seminyak_preprocessed.csv', index=False, encoding='utf-8')

In [2]:
#TF-IDF
df = pd.read_csv('dataset_preprocessed.csv')

In [6]:
# df = pd.DataFrame(np.random.randint(0.0,100.0,size=(10,1)),
#               index=range(10,20),
#               columns=['review'],
#               dtype='str')
corpus = df['review'].values.astype('U')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(stop_words='english')
resp = vec.fit_transform(corpus)
print(resp)

  (0, 5811)	0.10286300090343525
  (0, 6644)	0.22367809590360443
  (0, 10952)	0.09694992051264838
  (0, 3975)	0.2751462137393001
  (0, 1601)	0.255235627871483
  (0, 8658)	0.2751462137393001
  (0, 6009)	0.22891526502714843
  (0, 7657)	0.2261774893103979
  (0, 14857)	0.255235627871483
  (0, 8756)	0.255235627871483
  (0, 9475)	0.2634992676392385
  (0, 2777)	0.2391607183700504
  (0, 3173)	0.526998535278477
  (0, 1041)	0.255235627871483
  (0, 2282)	0.10040229023366344
  (1, 14024)	0.1412880865631301
  (1, 9958)	0.24440233826254834
  (1, 11102)	0.15960011042939204
  (1, 12816)	0.1881133441892455
  (1, 6886)	0.22258352055554814
  (1, 10437)	0.24559919752520354
  (1, 1529)	0.20372316423715034
  (1, 10555)	0.27437597389717383
  (1, 4234)	0.203599057996276
  (1, 13374)	0.13829346369981993
  :	:
  (10632, 12136)	0.21637347407168817
  (10632, 8325)	0.11707967582241233
  (10632, 15354)	0.10326914803105286
  (10632, 5364)	0.08956985714994037
  (10632, 10608)	0.08055901906027649
  (10632, 9080)	0.0844

In [10]:
vec.get_feature_names()

['000',
 '0000',
 '000400',
 '000idr',
 '000rp',
 '000s',
 '0045',
 '04',
 '0430',
 '05',
 '0500',
 '053',
 '0615',
 '0700',
 '085935102585',
 '0900',
 '0r',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000000',
 '10000000',
 '100000010',
 '100000idr',
 '100000ir',
 '100000k',
 '100000rp',
 '1000us',
 '100150k',
 '100200',
 '100k',
 '100kthe',
 '100m',
 '100rupes',
 '1010',
 '1011',
 '1011am',
 '1015',
 '1020',
 '10216',
 '1030',
 '103011',
 '1030ish',
 '104f',
 '105000',
 '10am',
 '10am2pmthe',
 '10ft',
 '10k',
 '10km',
 '10kyes',
 '10min',
 '10mins',
 '10mt',
 '10nz',
 '10pm',
 '10pmheaps',
 '10pmsit',
 '10th',
 '10x',
 '11',
 '110',
 '11000',
 '1100000',
 '1100am',
 '1100pm',
 '110k',
 '1110',
 '112',
 '115',
 '11am',
 '11k',
 '11pm',
 '11th',
 '12',
 '120',
 '120us',
 '121230',
 '122pm',
 '1230',
 '1230pm',
 '12km',
 '12nn330pm',
 '12pm',
 '12thfeb',
 '13',
 '1300',
 '1310',
 '137',
 '14',
 '1400',
 '140000',
 '1400000',
 '14a',
 '14km',
 '14kms',
 '14th',
 '15',
 '150',
 '1500'

In [11]:
df.head()

,Date,Content,review,scores,compound,sentiment
0,17-Aug,Best American Cheese Burger with oldies milksh...,best american cheese burger oldie milkshake va...,"{'neg': 0.0, 'neu': 0.569, 'pos': 0.431, 'comp...",0.8908,1
1,15-Dec,One of the beaches that are not full of touris...,one beach full tourist sure usually empty come...,"{'neg': 0.053, 'neu': 0.534, 'pos': 0.412, 'co...",0.9022,1
2,12-Jul,"the beach is clear and not so crowded, and als...",clear crowd people tide horse seller always bo...,"{'neg': 0.218, 'neu': 0.545, 'pos': 0.236, 'co...",0.0516,1
3,14-Nov,Crowded beach but worth to be out there to cat...,crowd worth catch wave best time sunset totall...,"{'neg': 0.0, 'neu': 0.541, 'pos': 0.459, 'comp...",0.8797,1
4,19-Dec,"If you’re looking for the idyllic, white sand ...",youre look idyllic white sand turquoise blue w...,"{'neg': 0.081, 'neu': 0.734, 'pos': 0.186, 'co...",0.5116,1
